# Form Recognition in Spark Ocr

End-to-end form recognition using VisualDocumentNerV2 and FormRelationExtractor.

## Install spark-ocr python packge
Need to specify:
- secret
- license
- aws credentials

In [1]:
secret = ""
license = ""
version = secret.split("-")[0]
spark_ocr_jar_path = "../../../target/scala-2.12/"
AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = ""

import os
if AWS_ACCESS_KEY_ID != "":
    os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    
if license:
    os.environ['JSL_OCR_LICENSE'] = license

In [2]:
import pyspark
import sparkocr
import json
import os

from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
import pyspark.sql.functions as f

from sparkocr.transformers import *
from sparkocr.utils import display_images
from sparkocr.enums import *

## Start spark session

In [3]:
from sparkocr import start
from pyspark import SparkConf

spark = start(secret, jar_path = spark_ocr_jar_path)
spark

Spark version: 3.3.0
Spark NLP version: 4.0.0
Spark NLP for Healthcare version: 4.0.0
Spark OCR version: 4.0.2rc1



## Load PDF, convert to image for rendering on notebook

In [ ]:
import pkg_resources
test_pdfs_path = './digital_forms/*.pdf'
bin_df = spark.read.format("binaryFile").load(test_pdfs_path)
display_images(PdfToImage().transform(bin_df))

## Define pipeline

In [ ]:
from sparkocr.transformers import PdfToForm
pdf_to_form = PdfToForm()\
    .setOutputCol("form")


pipeline = PipelineModel(stages=[
    pdf_to_form
])

## Call pipeline

In [ ]:
results = pipeline.transform(bin_df).cache()

In [ ]:
results.printSchema()

## Show resulst

In [ ]:
results.select(f.explode("form").alias('kvs')).\
select(f.col("kvs.key"), f.col("kvs.value"), f.col("kvs.coordinate")).\
show(20, False)